In [1]:
# Import Libraries, Modules & Dependencies
import panel as pn
import requests

# Initialize Panel Resources
pn.extension()

# Get CoinGecko coins list
def get_coingecko_coins():
    url = 'https://api.coingecko.com/api/v3/coins/list'
    response = requests.get(url)
    data = response.json()
    coin_dict = {coin['symbol'].upper(): coin['id'] for coin in data}
    return coin_dict

# Fetch the coin list and store it in coin_dict
coin_dict = get_coingecko_coins()

# Define a list of fiat currency codes
currencies = ['USD', 'EUR', 'JPY', 'GBP', 'AUD', 'CAD', 'CHF', 'CNY', 'SEK', 'NZD', 'SGD']

# Get price from CoinGecko - conditions to identify and store accordingly the inputs of from and to currency as they are in the currencies list or the coin_dict
def get_price(from_currency, to_currency):
    
    # Checking if the `from_currency` is present in `coin_dict`. 
    # If it is, the value of the corresponding key is assigned to `from_currency_id`.
    # If it isn't, the lowercase version of `from_currency` is assigned to `from_currency_id`.
    if from_currency in coin_dict:
        from_currency_id = coin_dict[from_currency]
    else:
        from_currency_id = from_currency.lower()
        
    # Checking if the `to_currency` is present in `coin_dict`.
    # If it is, the value of the corresponding key is assigned to `to_currency_id`.
    # If it isn't, the lowercase version of `to_currency` is assigned to `to_currency_id`.    
    if to_currency in coin_dict:
        to_currency_id = coin_dict[to_currency]
    else:
        to_currency_id = to_currency.lower()
        
    # Checking if both `from_currency` and `to_currency` are in `currencies`. 
    # If they are, the function fetches conversion rates for these currencies from the CoinGecko API.
    if from_currency in currencies and to_currency in currencies:
        vs_currency = from_currency_id
        usd_currency = 'usd'
        url = f'https://api.coingecko.com/api/v3/simple/price?ids={usd_currency}&vs_currencies={vs_currency}'
        response = requests.get(url)
        data = response.json()
        from_currency_rate = data[usd_currency][vs_currency]

        vs_currency = to_currency_id
        url = f'https://api.coingecko.com/api/v3/simple/price?ids={usd_currency}&vs_currencies={vs_currency}'
        response = requests.get(url)
        data = response.json()
        to_currency_rate = data[usd_currency][vs_currency]
        
        # The conversion rate is calculated as the ratio of `to_currency_rate` to `from_currency_rate`.
        conversion_rate = to_currency_rate / from_currency_rate
        
    # Checking if `from_currency` is in `currencies`.
    # If it is, the function fetches the conversion rate from the CoinGecko API.
    elif from_currency in currencies:
        vs_currency = from_currency.lower()
        url = f'https://api.coingecko.com/api/v3/simple/price?ids={to_currency_id}&vs_currencies={vs_currency}' 
        response = requests.get(url)
        data = response.json()
        conversion_rate = 1 / data[to_currency_id][vs_currency]
        
    # Checking if `to_currency` is in `currencies`.
    # If it is, the function fetches the conversion rate from the CoinGecko API.
    elif to_currency in currencies:
        vs_currency = to_currency.lower()
        url = f'https://api.coingecko.com/api/v3/simple/price?ids={from_currency_id}&vs_currencies={vs_currency}'
        response = requests.get(url)
        data = response.json()
        conversion_rate = data[from_currency_id][vs_currency]
        
    # If neither `from_currency` nor `to_currency` are in `currencies`, 
    # the function fetches the conversion rates for these currencies against USD from the CoinGecko API.
    else:
        vs_currency = 'usd'
        url = f'https://api.coingecko.com/api/v3/simple/price?ids={from_currency_id},{to_currency_id}&vs_currencies={vs_currency}'
        response = requests.get(url)
        data = response.json()
        
        # Fetches the exchange rate for the 'from_currency' against USD.
        from_currency_rate = data[from_currency_id][vs_currency]
        
        # Fetches the exchange rate for the `to_currency` against USD.
        to_currency_rate = data[to_currency_id][vs_currency]
        
        # The conversion rate is calculated as the ratio of `from_currency_rate` to `to_currency_rate`.
        conversion_rate = from_currency_rate / to_currency_rate

    return conversion_rate

# Create widgets
from_currency_widget = pn.widgets.AutocompleteInput(name='From: Fiat or Crypto', options=list(coin_dict.keys()) + currencies, value='AUD')
to_currency_widget = pn.widgets.AutocompleteInput(name='To: Crypto or Fiat', options=list(coin_dict.keys()) + currencies, value='XRP')
amount_widget = pn.widgets.FloatInput(name='Amount', value=1.0)

# Modify the convert_currencies function to use CoinGecko API for both crypto and fiat conversions
def convert_currencies(from_currency, to_currency, amount):
    if from_currency == to_currency:
        return amount
    else:
        price = get_price(from_currency, to_currency)
        return round(amount * price, 8)

# Update the conversion result when the button is clicked
def update_conversion_result(event):
    from_currency = from_currency_widget.value
    to_currency = to_currency_widget.value
    amount = amount_widget.value
    result = convert_currencies(from_currency, to_currency, amount)
    conversion_result_widget.value = f'{amount} {from_currency.upper()} = {result} {to_currency.upper()}' #Swapped

# Create result widget and button with cell-like appearance
conversion_result_widget = pn.widgets.TextInput(name='Result', value='', disabled=True)
convert_button = pn.widgets.Button(name='Convert', button_type='primary', background='#ADD8E6')
convert_button.on_click(update_conversion_result)

# Create the currency converter panel
currency_converter = pn.Column("Dynamic Currency Converter", from_currency_widget, to_currency_widget, amount_widget, convert_button, conversion_result_widget)

# Display the dashboard in a pop-up window
#dashboard = pn.Tabs(('Converter', currency_converter))
#dashboard.show()

# Converter tab ends here ------------------------------------------------------------------------------------------

# Alerts tab starts here -------------------------------------------------------------------------------------------

# Import Libraries, Modules & Dependencies
from twilio.rest import Client
import time
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Create a new pn.pane.Markdown object for displaying the success message
success_message = pn.pane.Markdown("", width=300, height=50)

# Your Twilio API credentials
account_sid = os.getenv('TWILIO_ACCOUNT_SID')
auth_token = os.getenv('TWILIO_AUTH_TOKEN')
from_phone_number = os.getenv('TWILIO_PHONE_NUMBER')

# Create a function to send text messages using the Twilio API
def send_sms(message, to_phone_number):
    client = Client(account_sid, auth_token)
    client.messages.create(
        body=message,
        from_=from_phone_number,
        to=to_phone_number
    )

# Create a function that periodically checks the current price of the selected cryptocurrency using the CoinGecko API. If the price reaches the target, call the send_sms function to send the text message to the user
def check_price_and_send_sms(selected_crypto, price_target, phone_number):
    while True:
        current_price = get_price(selected_crypto, 'usd')
        if current_price >= float(price_target):
            send_sms(f"{selected_crypto} has reached your target price of ${price_target} AUD!", phone_number)
            break
        time.sleep(60)

# Create the input widgets for the "Alerts" tab
alert_crypto_widget = pn.widgets.AutocompleteInput(name='Select Crypto:', options=list(coin_dict.keys()), value='XRP')
price_target_widget = pn.widgets.TextInput(name='Price Target:', placeholder='Enter your price target')
phone_number_widget = pn.widgets.TextInput(name='Phone Number:', placeholder='+61452124288')
send_alert_button = pn.widgets.Button(name='Schedule Alert', button_type='primary')

# Create a function to handle the click event on the "Send Alert" button
def on_send_alert_click(event):
    selected_crypto = alert_crypto_widget.value
    price_target = price_target_widget.value
    phone_number = phone_number_widget.value

    # Display the success message
    success_message.object = "✔ Alert scheduled successfully!"

    # You can use threading to run the function without blocking the dashboard
    import threading
    thread = threading.Thread(target=check_price_and_send_sms, args=(selected_crypto, price_target, phone_number))
    thread.start()

# Add the click event handler to the "Send Alert" button
send_alert_button.on_click(on_send_alert_click)

# Create the "Alerts" tab layout and add it to the dashboard
alerts_tab = pn.Column(
    "Crypto Price Alert",
    alert_crypto_widget,
    price_target_widget,
    phone_number_widget,
    send_alert_button,
    success_message
)

dashboard = pn.Tabs(('Converter', currency_converter), ('Alerts', alerts_tab))
dashboard.show()

# End of Alerts tab code -----------------------------------------------------------------------------

Launching server at http://localhost:60274
